# 🚀 Mô Hình Dự Báo Giá Bitcoin Với BiLSTM

Notebook này hướng dẫn bạn từng bước để xây dựng và huấn luyện mô hình dự báo giá Bitcoin.

---

## 📋 Checklist

- [ ] Bước 1: Lấy dữ liệu từ Binance
- [ ] Bước 2: Xử lý dữ liệu
- [ ] Bước 3: Xây dựng model BiLSTM
- [ ] Bước 4: Training model
- [ ] Bước 5: Đánh giá & Vẽ biểu đồ

---

## 💡 Tips

1. Chạy từng cell theo thứ tự từ trên xuống dưới
2. Đọc comments trong code để hiểu
3. Nếu cần giải thích chi tiết, xem `docs/` folder
4. Nghỉ giải lao nếu cảm thấy ngợp!

## 📦 0. Setup & Cấu Hình

Cấu hình TensorFlow và import các thư viện cần thiết.

In [ ]:
# Import thư viện
import numpy as np
import polars as pl
import tensorflow as tf
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import modules từ project
# Note: Notebook đang chạy từ notebooks/, cần up 1 cấp để tới project root
import sys
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from utils.runtime import configure_tensorflow_runtime, print_tensorflow_info, set_random_seed

# Cố định seed để tái lập kết quả (nếu SEED < 0 thì bỏ qua)
try:
    set_random_seed(SEED, deterministic=True)
except NameError:
    # Nếu cell cấu hình chưa chạy, fallback seed mặc định
    set_random_seed(42, deterministic=True)

# Cấu hình TensorFlow cho CPU AMD
configure_tensorflow_runtime(
    intra_op_threads=12,
    inter_op_threads=2,
    enable_xla=True
)

# In thông tin TensorFlow
print_tensorflow_info()

## 🔧 1. Cấu Hình Tham Số

Thay đổi các tham số tùy ý:

In [ ]:
# ==================== CẤU HÌNH ====================

# Reproducibility
SEED = 42  # Đổi seed để chạy thử nghiệm; đặt <0 để không cố định

# Data parameters (CSV local)
DATA_PATH = "data/btc_1d_data_2018_to_2025.csv"  # Đổi sang 4h nếu muốn: data/btc_4h_data_2018_to_2025.csv
LIMIT = 1500                                      # Lấy N dòng cuối (<=0 = lấy tất cả)
REFRESH_CACHE = False                             # True = đọc lại CSV gốc, False = dùng cache normalized

def infer_timeframe_from_filename(path: str) -> str:
    name = path.lower()
    if "4h" in name:
        return "4h"
    if "1d" in name:
        return "1d"
    return "unknown"

TIMEFRAME = infer_timeframe_from_filename(DATA_PATH)  # Hiển thị timeframe dựa vào tên file

# Preprocessing parameters
WINDOW_SIZE = 100            # Số nến nhìn lại (sliding window)
FEATURES = ["close"]       # Features sử dụng

# Model parameters
LSTM_UNITS = [64, 32]       # Số units cho mỗi LSTM layer
DROPOUT_RATE = 0.2         # Dropout rate

# Training parameters
EPOCHS = 20                 # Số epochs
BATCH_SIZE = 32             # Batch size
EARLY_STOPPING_PATIENCE = 5 # Số epochs chờ trước khi dừng

# Print cấu hình
print("="*60)
print("⚙️  CẤU HÌNH")
print("="*60)
print(f"Seed: {SEED}")
print(f"Data path: {DATA_PATH}")
print(f"Timeframe (from filename): {TIMEFRAME}")
print(f"Limit (tail): {LIMIT}")
print(f"Window size: {WINDOW_SIZE}")
print(f"Features: {FEATURES}")
print(f"LSTM units: {LSTM_UNITS}")
print(f"Dropout: {DROPOUT_RATE}")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print("="*60 + "\n")

---

## 📥 BƯỚC 1: ĐỌC DỮ LIỆU CSV (LOCAL)

### Giải thích:
- Đọc dữ liệu giá từ file CSV local (mặc định: `data/btc_1d_data_2018_to_2025.csv`)
- Chuẩn hoá về DataFrame với: datetime, open, high, low, close, volume
- Cache (optional) file CSV đã chuẩn hoá để lần sau đọc nhanh hơn

In [ ]:
from step1_data import fetch_binance_data

# Đọc dữ liệu (CSV local)
df = fetch_binance_data(
    data_path=DATA_PATH,
    timeframe=TIMEFRAME,
    limit=LIMIT,
    save_cache=not REFRESH_CACHE
)

# In 5 dòng đầu tiên
print("\n📊 5 dòng đầu tiên của dữ liệu:")
print(df.head())

# Thống kê cơ bản
print("\n📊 Thống kê dữ liệu:")
print(df.describe())

# Đánh dấu checklist
print("\n✅ Bước 1 hoàn thành!")

### Vẽ biểu đồ lịch sử giá

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

plt.figure(figsize=(14, 6))
plt.plot(df['datetime'], df['close'], linewidth=2, color='#2E86AB', label='Giá đóng nến')
plt.title(f'Lịch sử giá Bitcoin ({TIMEFRAME})', fontsize=16, fontweight='bold')
plt.xlabel('Thời gian', fontsize=12)
plt.ylabel('Giá (USD)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
plt.tight_layout()
plt.show()

---

## 🔨 BƯỚC 2: XỬ LÝ DỮ LIỆU

### Giải thích:
- **Scaling**: Đưa dữ liệu về khoảng [0, 1] để model học tốt hơn
- **Chống data leakage**: scaler được **fit chỉ trên tập train**, sau đó mới transform val/test
- **Sliding Window**: Tạo sequences (60 ngày trước → dự đoán ngày tiếp theo)
- **Split Data**: Chia thành train (80%), val (10%), test (10%)

In [ ]:
from step2_preprocessing import prepare_data_for_lstm

# Pipeline xử lý dữ liệu hoàn chỉnh
data_dict = prepare_data_for_lstm(
    df=df,
    features=FEATURES,
    window_size=WINDOW_SIZE,
    scaler_type='minmax'
)

# Lấy các biến
X_train = data_dict['X_train']
y_train = data_dict['y_train']
X_val = data_dict['X_val']
y_val = data_dict['y_val']
X_test = data_dict['X_test']
y_test = data_dict['y_test']
scaler = data_dict['scaler']

# Đánh dấu checklist
print("\n✅ Bước 2 hoàn thành!")

### Kiểm tra shapes của dữ liệu

In [ ]:
print("\n" + "="*60)
print("📊 SHAPES CỦA DỮ LIỆU")
print("="*60)
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val:   {X_val.shape}")
print(f"y_val:   {y_val.shape}")
print(f"X_test:  {X_test.shape}")
print(f"y_test:  {y_test.shape}")
print("="*60 + "\n")

---

## 🧠 BƯỚC 3: XÂY DỰNG MODEL BiLSTM

### Giải thích:
- **BiLSTM**: LSTM hai chiều (nhìn cả quá khứ và tương lai)
- **Dropout**: Bỏ ngẫu nhiên neurons để tránh overfitting
- **Dense layers**: Kết hợp features để đưa ra dự đoán

In [ ]:
from step3_model import build_bilstm_model, print_model_summary

# Xây dựng model
input_shape = (WINDOW_SIZE, len(FEATURES))
model = build_bilstm_model(
    input_shape=input_shape,
    lstm_units=LSTM_UNITS,
    dropout_rate=DROPOUT_RATE,
    dense_units=[16],
    output_units=1
)

# In thông tin model
print_model_summary(model)

# Đánh dấu checklist
print("\n✅ Bước 3 hoàn thành!")

---

## 🏋️ BƯỚC 4: TRAINING MODEL

### Giải thích:
- **ModelCheckpoint**: Lưu lại model tốt nhất
- **EarlyStopping**: Dừng nếu val_loss không giảm
- **ReduceLROnPlateau**: Giảm learning rate nếu không cải thiện

In [ ]:
from step4_training import train_model

# Training
train_result = train_model(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    early_stopping_patience=EARLY_STOPPING_PATIENCE
)

# Lấy training history
history = train_result['history']

# Metadata training (để đưa vào report)
best_epoch = train_result.get('best_epoch')
best_val_loss = train_result.get('best_val_loss')
train_seconds = train_result.get('train_seconds')
checkpoint_path = str(train_result.get('checkpoint_path')) if train_result.get('checkpoint_path') is not None else None

print(f"\n📌 Best epoch: {best_epoch}")
print(f"📌 Best val_loss: {best_val_loss}")
print(f"📌 Training time (s): {train_seconds}")

# Đánh dấu checklist
print("\n✅ Bước 4 hoàn thành!")

### Vẽ biểu đồ training history

In [ ]:
from step5_visualization import plot_training_history

plot_training_history(history)

---

## 📊 BƯỚC 5: ĐÁNH GIÁ & VẼ BIỂU ĐỒ

### Giải thích:
- **MAE**: Sai số trung bình tuyệt đối (USD)
- **RMSE**: Căn bậc 2 của sai số bình phương trung bình (USD)
- **MAPE**: Sai số phần trăm trung bình (%)

In [ ]:
from step4_training import evaluate_model, print_sample_predictions, calculate_direction_accuracy
from step5_visualization import plot_predictions, plot_all_in_one

# Đánh giá trên test set
eval_result = evaluate_model(
    model=model,
    X_test=X_test,
    y_test=y_test,
    scaler=scaler,
    return_predictions=True
)

# Lấy dự đoán và giá trị thật
y_true = eval_result['y_true']
y_pred = eval_result['predictions']

In [ ]:
# In một số ví dụ dự đoán
print_sample_predictions(y_true, y_pred, n_samples=10)

In [ ]:
# Tính độ chính xác xu hướng
direction_accuracy = calculate_direction_accuracy(y_true, y_pred)

# Lưu vào eval_result để report/metrics.json có thêm thông tin
eval_result['direction_accuracy'] = float(direction_accuracy)

### Vẽ biểu đồ predictions vs actual

In [ ]:
plot_predictions(y_true, y_pred)

### Vẽ biểu đồ tổng hợp (all-in-one)

In [ ]:
plot_all_in_one(history, y_true, y_pred)

---

## 💾 LƯU KẾT QUẢ

Tất cả kết quả được lưu vào thư mục `reports/notebook/`

In [ ]:
from utils import create_results_folder, save_markdown_report, save_config, save_metrics

# Tạo folder kết quả
results_folder = create_results_folder(run_type="notebook")
print(f"📁 Folder kết quả: {results_folder}\n")

# Tạo suffix cho tên file
timestamp_suffix = results_folder.name.replace('BiLSTM_', '')

# Vẽ và lưu biểu đồ
plot_history_file = results_folder / f"training_history_{timestamp_suffix}.png"
plot_predictions_file = results_folder / f"predictions_{timestamp_suffix}.png"
plot_all_in_one_file = results_folder / f"all_in_one_{timestamp_suffix}.png"

plot_training_history(history, save_path=str(plot_history_file))
plot_predictions(y_true, y_pred, save_path=str(plot_predictions_file))
plot_all_in_one(history, y_true, y_pred, save_path=str(plot_all_in_one_file))

# Metadata dữ liệu
try:
    data_rows = len(df)
    data_start = str(df['datetime'].iloc[0])
    data_end = str(df['datetime'].iloc[-1])
except Exception:
    data_rows, data_start, data_end = None, None, None

# Metadata split
train_samples = len(X_train)
val_samples = len(X_val)
test_samples = len(X_test)
scaler_type = 'minmax'

# Chuẩn bị config và metrics
config_dict = {
    'symbol': SYMBOL,
    'timeframe': TIMEFRAME,
    'limit': LIMIT,
    'data_rows': data_rows,
    'data_start': data_start,
    'data_end': data_end,
    'window_size': WINDOW_SIZE,
    'features': FEATURES,
    'scaler_type': scaler_type,
    'train_samples': train_samples,
    'val_samples': val_samples,
    'test_samples': test_samples,
    'seed': SEED,
    'lstm_units': LSTM_UNITS,
    'dropout_rate': DROPOUT_RATE,
    'epochs': EPOCHS,
    'batch_size': BATCH_SIZE,
    'early_stopping_patience': EARLY_STOPPING_PATIENCE,
    'best_epoch': best_epoch,
    'best_val_loss': best_val_loss,
    'train_seconds': train_seconds,
    'checkpoint_path': checkpoint_path,
}

plots_dict = {
    'training_history': timestamp_suffix,
    'predictions': timestamp_suffix,
    'all_in_one': timestamp_suffix
}

# Lưu báo cáo
save_markdown_report(
    folder_path=results_folder,
    config=config_dict,
    metrics=eval_result,
    history=history.history,
    plots=plots_dict
)
save_config(results_folder, config_dict)
save_metrics(results_folder, eval_result)

# Đánh dấu checklist
print("\n✅ Bước 5 hoàn thành!")

---

## 🎉 HOÀN THÀNH!

### Checklist:
- [x] Bước 1: Lấy dữ liệu từ Binance
- [x] Bước 2: Xử lý dữ liệu
- [x] Bước 3: Xây dựng model BiLSTM
- [x] Bước 4: Training model
- [x] Bước 5: Đánh giá & Vẽ biểu đồ

### Kết quả:
- Báo cáo Markdown: `reports/notebook/BiLSTM_YYYYMMDD_HHMMSS/results_*.md`
- Biểu đồ: Các file PNG trong cùng folder
- Config & Metrics: File JSON trong cùng folder

### Tiếp theo:
- Thử thay đổi các tham số (window size, LSTM units, v.v.)
- Thêm các features khác (volume, open, high, low)
- Thử timeframe khác (4h, 1h)

**Chúc bạn thành công! 🚀**